# **Automated Trading Operations**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
import scipy.stats as scs
import pickle
import tpqoa

ModuleNotFoundError: No module named 'tpqoa'

### **Capital Management 101: Kelly Criterion**

-The Kelly criterion (also known as the scientific gambling method) is a formula which helps investors and gamblers size their bets

-It is valid when the expected returns are known...

-The Kelly criterion is found by optimizing the expected value of the logarithm of wealth, equivalent to maximizing the expected geometric growth rate

$$ K\% = W - \frac{1-W}{R} $$

Where $K$ is the Kelly percentage, $W$ is the winning probability, and $R$ is the win/loss ratio (average gain over average loss. *fractional odds* in the UK)

The investment relative form is:

$$ f^* = \frac{p}{a} - \frac{q}{b} $$

Where $f^*$ is the fraction of the assets to position in the security, $p$ is the probability that the security will increase in value, $q$ is the probability that the security will decrease in value, $a$ is the fraction that is lost in a negative outcome (i.e. the security price falling 1%, then $a$=.01), and $b$ is the fraction that is gained in a positive outcome

-Note that these quantities are not known, but estimated! So it is recommended for risk averse investors to invest a certain fraction of Kelly. 

-For stock investments, assume geometric Brownian motion (also known as exponential brownian motion) and maximize the single period log returns w.r.t fraction invested. You arrive at:

$$ f^* = \frac{\mu - r}{\sigma^2} $$

Where $\mu$ is the average log gross return, $r$ is the risk-free rate, and $\sigma$ is the std of the gross returns. See Stock Investments section of: https://en.wikipedia.org/wiki/Kelly_criterion

In [57]:
%time
api = tpqoa.tpqoa('../../data/keys.cfg')
instrument = 'EUR_USD'

raw = api.get_history(instrument,
                    start='2020-06-08',
                    end='2020-06-13',
                    granularity='M10',
                    price='M')
raw.tail()

CPU times: total: 0 ns
Wall time: 0 ns


,o,h,l,c,volume,complete
time,,,,,,
2020-06-12 20:10:00,1.12572,1.12593,1.12532,1.12568,221,True
2020-06-12 20:20:00,1.12569,1.12578,1.12532,1.12558,163,True
2020-06-12 20:30:00,1.12560,1.12573,1.12534,1.12543,192,True
2020-06-12 20:40:00,1.12544,1.12594,1.12528,1.12542,219,True
2020-06-12 20:50:00,1.12544,1.12624,1.12541,1.12554,296,True


In [58]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 701 entries, 2020-06-08 00:00:00 to 2020-06-12 20:50:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   o         701 non-null    float64
 1   h         701 non-null    float64
 2   l         701 non-null    float64
 3   c         701 non-null    float64
 4   volume    701 non-null    int64  
 5   complete  701 non-null    bool   
dtypes: bool(1), float64(4), int64(1)
memory usage: 33.5 KB


### **Bid-Ask Spread Calculation**

In [59]:
# see https://en.wikipedia.org/wiki/Bid%E2%80%93ask_spread for calculations of bid-ask spread

bid = api.get_history(instrument,
                    start='2020-06-08',
                    end='2020-06-13',
                    granularity='M10',
                    price='B')
ask = api.get_history(instrument,
                    start='2020-06-08',
                    end='2020-06-13',
                    granularity='M10',
                    price='A')
((ask['l'] - bid['h'])  / raw['h']).mean()

-0.0005446687480155511

In [60]:
# specify the average bid ask spread
# not sure how hilpisch arrived at this lower value than the one above
spread = .00012

# mean closing price
mean = raw['c'].mean()

# average proportional transaction cost
ptc = spread / mean

### **Feature Engineering**

In [61]:
# market movement prediction strategy

data = pd.DataFrame(raw['c'])
data.columns = [instrument,]

# feature engineering
# create some moving averages, stds, and min/max
# keep in mind, you can use numpy functionality when all of the data in a
# pandas dataframe are numeric! 

# log gross returns (you lose first n number of shifts of the dataframe rows)
data['log_returns'] = np.log(data / data.shift(1))
window = 20
# rolling std (you lose first n window size of the dataframe rows)
data['vol'] = data['log_returns'].rolling(window).std()
# sign of rolling mean
data['mom'] = np.sign(data['log_returns'].rolling(window).mean())
# rolling mean of the price (a validation of technical analysis)
data['sma'] = data[instrument].rolling(window).mean()
# rolling min and max
data['min'] = data[instrument].rolling(window).min()
data['max'] = data[instrument].rolling(window).max()
data.dropna(inplace=True)
data.head()

# lag the data
lags = 6
features = ['log_returns', 'vol', 'mom', 'sma', 'min', 'max']

cols = []
for f in features:
    for lag in range(1, lags + 1):
        col = f"{f}_lag_{lag}"
        data[col] = data[f].shift(lag)
        cols.append(col)

data.dropna(inplace=True)
# define the target variable
data['direction'] = np.where(data['log_returns'] > 0, 1, -1)

data.head()

,EUR_USD,log_returns,vol,mom,sma,min,max,log_returns_lag_1,log_returns_lag_2,log_returns_lag_3,...,min_lag_4,min_lag_5,min_lag_6,max_lag_1,max_lag_2,max_lag_3,max_lag_4,max_lag_5,max_lag_6,direction
time,,,,,,,,,,,,,,,,,,,,,
2020-06-08 04:20:00,1.12897,-0.000115,0.000303,-1.0,1.129240,1.12874,1.12962,0.000097,0.000018,-0.000452,...,1.12874,1.12874,1.12874,1.12980,1.12980,1.13071,1.13071,1.13117,1.13117,-1
2020-06-08 04:30:00,1.12900,0.000027,0.000254,1.0,1.129242,1.12874,1.12962,-0.000115,0.000097,0.000018,...,1.12874,1.12874,1.12874,1.12962,1.12980,1.12980,1.13071,1.13071,1.13117,1
2020-06-08 04:40:00,1.12884,-0.000142,0.000240,-1.0,1.129215,1.12874,1.12962,0.000027,-0.000115,0.000097,...,1.12874,1.12874,1.12874,1.12962,1.12962,1.12980,1.12980,1.13071,1.13071,-1
2020-06-08 04:50:00,1.12888,0.000035,0.000203,1.0,1.129222,1.12884,1.12962,-0.000142,0.000027,-0.000115,...,1.12874,1.12874,1.12874,1.12962,1.12962,1.12962,1.12980,1.12980,1.13071,1
2020-06-08 05:00:00,1.12870,-0.000159,0.000187,-1.0,1.129199,1.12870,1.12962,0.000035,-0.000142,0.000027,...,1.12874,1.12874,1.12874,1.12962,1.12962,1.12962,1.12962,1.12980,1.12980,-1


In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 675 entries, 2020-06-08 04:20:00 to 2020-06-12 20:50:00
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EUR_USD            675 non-null    float64
 1   log_returns        675 non-null    float64
 2   vol                675 non-null    float64
 3   mom                675 non-null    float64
 4   sma                675 non-null    float64
 5   min                675 non-null    float64
 6   max                675 non-null    float64
 7   log_returns_lag_1  675 non-null    float64
 8   log_returns_lag_2  675 non-null    float64
 9   log_returns_lag_3  675 non-null    float64
 10  log_returns_lag_4  675 non-null    float64
 11  log_returns_lag_5  675 non-null    float64
 12  log_returns_lag_6  675 non-null    float64
 13  vol_lag_1          675 non-null    float64
 14  vol_lag_2          675 non-null    float64
 15  vol_lag_3          675 non-null    fl

In [63]:
n_estimators=15
random_state=100
max_depth=2
min_samples_leaf=15
subsample=0.33

dtc = DecisionTreeClassifier(random_state=random_state,
                            max_depth=max_depth,
                            min_samples_leaf=min_samples_leaf)

# default base estimator of adaboost is the decision tree
model = AdaBoostClassifier(base_estimator=dtc,
                          n_estimators=n_estimators,
                          random_state=random_state)

# train, test split (70%)
split = int(len(data) * 0.7)

train = data.iloc[:split].copy()

mu, std = train.mean(), train.std()

# normalize training dataset
train_ = (train - mu) / std

# lets just train on the lagged features. 
# take non normalized target variable 
# because the normalization turns it into 
# a continuous variable
model.fit(train_[cols], train['direction'])


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2,
                                                         min_samples_leaf=15,
                                                         random_state=100),
                   n_estimators=15, random_state=100)

In [70]:
# sklearn classifiers don't necessarily care if your labels are {0,1} or {-1,1}
model.predict(train_[cols])

array([-1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,
        1,  1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1,
       -1, -1,  1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1,
        1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1,
        1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,
        1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,
       -1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1,
        1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1, -1,  1, -1, -1,
       -1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  1,  1, -1, -1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1,
       -1, -1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,
        1,  1,  1,  1,  1

In [ ]:
# lets look at the fraction of correctly identified market movements 
# (From a financial trading standpoint, this is the hit ratio: the number
# of winning trades compared to all trades)
# in the context of Kelly, greater than 50% indicates an edge
# the hit ratio is only one side of the coin of a successful trading bot
# the other concerns geting the transaction costs right, and the largest 
# market movements right! 

# on training set
print(f"Accuracy of boosting classifier on training set: {accuracy_score(train['direction'], model.predict(train_[cols]))}")

# on test set
test = data.iloc[split:].copy()

test_ = (test - mu) / std

print(f"Accuracy of boosting classifier on test set: {accuracy_score(test['direction'], model.predict(test_[cols]))}")

In [ ]:
# visualizing the trading strategy on the hold-out set

test['position'] = model.predict(test_[cols])
test['strategy'] = test['position'] * test['log_returns']

# the number of trades
sum(test['strategy'].diff() != 0)

# strategy minus transaction costs
test['strategy_tc'] = np.where(test['position'].diff() != 0,
                                test['strategy'] - ptc,
                                test['strategy'])

# plot
test[['log_returns', 'strategy', 'strategy_tc']].cumsum().apply(np.exp).plot(figsize=(10, 6))


### **Optimal Leverage**

In [ ]:
# first annualize the average of the log returns

# multiple by number of 10 minute intervals in that week (len(data)) then
# by the number of weeks in a year
mean = test[['log_returns', 'strategy_tc']].mean() * len(data) * 52
var = test[['log_returns', 'strategy_tc']].var() * len(data) * 52

print(f"Kelly criterion: \n{mean / var}")
print(f".5 Kelly criterion: \n{0.5 * mean / var}")

# thats a leverage of 50:1 which is doable on Oanda

In [ ]:
# plot strategies with the leverages

to_plot = ['log_returns', 'strategy_tc']

for lev in [10, 20, 30, 40, 50]:
    label = f"lev_strategy_tc_{lev}"
    # this is multplying the lever by log returns! Not exactly accurate I think
    # These returns are actually with respect to your equity, so I think it makes sense
    test[label] = test['strategy_tc'] * lev
    to_plot.append(label)

test[to_plot].cumsum().apply(np.exp).plot(figsize=(10, 6))



In [ ]:
# The largest dip at about 3:00 pm
test['lev_strategy_tc_50'].cumsum().apply(np.exp).min()

# so you lost 20% of your account equity ($1000)
# this would not trigger a margin call, I believe

In [ ]:
# when you actually do the cumulative returns at that minimum,
# its actually worse since in the calculation above, we multiplied
# the lever by the log returns (should be multiplied by gross returns)
# still don't think it would have triggered a margin call though. 
250000 * test['strategy_tc'].cumsum().apply(np.exp).min() - 250000

In [ ]:
# zooming in on the buy and hold and the non-levered strategy
# The effect of the transaction costs are probably underestimated too
# since they are also multipled by the log returns
test[['log_returns', 'strategy_tc']].cumsum().apply(np.exp).plot(figsize=(10, 6))

### **Risk Analysis**

In [ ]:
pd.set_option('display.max_rows', 100)
# lets look at maximum drawdown (largest loss (dip) after a recent high) and longest maximum drawdown (longest period that the strategy needs to get back to a recent high)

# we were actually analyzing the max drawdown right above to see what would happen with the margin call
# possibility

# lets assume that there are no adjustments with regard to the equity over time, no matter the performance

# this is truly your net worth, since the leveraged amount is a liability
equity = 5000
lev_strat = 'lev_strategy_tc_50'

risk = pd.DataFrame(test[lev_strat])

risk['equity'] = risk[lev_strat].cumsum().apply(np.exp) * equity
risk['cummax'] = risk['equity'].cummax()
risk['drawdown'] = risk['cummax'] - risk['equity']
print(f"Max drawdown for {lev_strat} is: {risk['drawdown'].max()} \n and this occurs at {risk['drawdown'].idxmax()}")
# as you can see by the plot below, this is due to the later drawdown
# when our equity is already trending up. The earlier drawdown is what 
# we're afraid of! 
risk.head()

In [ ]:
# plotting your net worth here
risk['equity'].plot(figsize=(10, 6))

In [ ]:
# longest drawdown time
# a new high is characterized by a drawdown of 0

temp = risk['drawdown'][risk['drawdown'] == 0]
# essentially subtract array of n = 2 : end highs and arragy of n = 1 : end - 1
periods = (temp.index[1:].to_pydatetime() - temp.index[:-1].to_pydatetime())
t_per = periods.max()
print(f"Longest drawdown is: {np.round(t_per.seconds / 60 / 60, 2)} hours")

risk[['equity', 'cummax']].plot(figsize=(10, 6))
plt.axvline(risk['drawdown'].idxmax(), c='r', alpha=0.5)

### **Value at Risk**

- when you want to look at the odds of losing greater than or equal to a certain amount at the left tail of the PnL distribution

- What are the worst case scenarios and what are their odds? 

- The left tail of the PnL distribution is highly unlikely, but what loss values occupy that space? What is the 1st percentile, 3rd percentile, and 5th percentile of your PnL density function? 

-e.g. 95% VaR of $1 million means there is a 5% chance of losing this amount or greater. is the odds of losing an amount greater than or equal to the value at the 5th percentile of the PnL PDF

In [ ]:
# percentiles
percs = [.01, .1, 1., 2.5, 5.0, 10.0]

risk['log_returns'] = np.log(risk['equity'] / risk['equity'].shift(1))

VaR = scs.scoreatpercentile(equity * risk['log_returns'], percs)

def print_var():
    print(f"Confidence Interval     Value-at-Risk")
    print(33 * '-')
    for pair in zip(percs, VaR):
        print(f"{100 - pair[0]}      {-pair[1]}")

print_var()


# this is not taking the lever into account!!! multiply the bottom
# VaR by 20!!!!!
# these are you worst possible losses in one bar (in this case 10 minutes)

In [ ]:
risk.head()

In [ ]:
# downsample from minute bars to hour bars
# take the last datapoint for the hour datapoints
hourly = risk.resample('1H', label='right').last()
# need to recalculate log returns for new time frequency
hourly['log_returns'] = np.log(hourly['equity'] /hourly['equity'].shift(1))
VaR = scs.scoreatpercentile(equity * hourly['log_returns'], percs)
print_var()
# again multiply bottom by lever!!!
# obviously, the worst case losses increase as you consider a longer time span

### **Save the Model**

In [ ]:
# once we're good with the backtesting, capital management, and risk management analysis,
# we can go ahead and save the model, as well as other import information 

algorithm = {'model': model, 'mu': mu, 'std': std}
# 'wb' means write to binary
pickle.dump(algorithm, open('../../data/algorithm.pkl', 'wb'))

### **Online Algorithm**

-In practice, when deploying the trading algorithm in financial markets, it must consume data piece by piece as it arrives to predict the direction of the market movement for the next time interval

-Now we are going to transform the *offline* algorithm into an *online* algorithm

  - To do this, we must address processing the *tick data* in real time, *resampling* to the appropriate bar length, *predicting* over the relevant time interval, and *ordering* given the current position and prediction (signal)

In [73]:
# load pickle file in read binary mode
algorithm = pickle.load(open('../../data/algorithm.pkl', 'rb'))

algorithm['model']

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2,
                                                         min_samples_leaf=15,
                                                         random_state=100),
                   n_estimators=15, random_state=100)

### **Validating Data Length After Resampling, Windowing, and Lagging**

Still have some unanswered questions

In [55]:
raw_investigate = raw.iloc[:91][['c']]
print(f"tail of raw is: {raw_investigate[-10:]}")
# look at resampling before lagging and windowing
raw_resample = raw_investigate.resample('1H', label='right') #.last().ffill().iloc[:-1]
print(f"tail of raw after resample and taking last: {raw_resample.last().tail()}")
print(f"tail of raw after resample, taking last, and ffill: {raw_resample.last().ffill().tail()}")

print(f"head of raw after resample and taking last: {raw_resample.last().head()}")
print(f"head of raw after resample, taking last, and ffill: {raw_resample.last().ffill().head()}")

# not sure what ffill is for!!!

tail of raw is:                            c
time                        
2020-06-08 13:30:00  1.13032
2020-06-08 13:40:00  1.13162
2020-06-08 13:50:00  1.13097
2020-06-08 14:00:00  1.13046
2020-06-08 14:10:00  1.13051
2020-06-08 14:20:00  1.13082
2020-06-08 14:30:00  1.12979
2020-06-08 14:40:00  1.12994
2020-06-08 14:50:00  1.13009
2020-06-08 15:00:00  1.13033
tail of raw after resample and taking last:                            c
time                        
2020-06-08 12:00:00  1.12764
2020-06-08 13:00:00  1.12940
2020-06-08 14:00:00  1.13097
2020-06-08 15:00:00  1.13009
2020-06-08 16:00:00  1.13033
tail of raw after resample, taking last, and ffill:                            c
time                        
2020-06-08 12:00:00  1.12764
2020-06-08 13:00:00  1.12940
2020-06-08 14:00:00  1.13097
2020-06-08 15:00:00  1.13009
2020-06-08 16:00:00  1.13033
head of raw after resample and taking last:                            c
time                        
2020-06-08 01:00:00  1.12976
202

In [48]:
raw_investigate = raw.iloc[:91][['c']]
print(f"tail of raw is: {raw_investigate.tail()}")
# look at resampling before lagging and windowing
raw_resample = raw_investigate.resample('1H', label='right').last().ffill().iloc[:-1]
print(f"length of raw_resample is {len(raw_resample)}")
print(f"tail after resampling {raw_resample.tail()}")

raw1 = raw_resample.copy()
print(f"length of raw1 is: {len(raw1)}")
raw1_mov = raw1['c'].rolling(14).mean()
print(f"length of raw1_mov is {len(raw1_mov)}")
print(f"data after moving average: \n {raw1_mov.tail()}")

# raw2 = raw[:15].copy()
# print(f"length of raw2 is: {len(raw2)}")
# raw2_lag = raw2['c'].shift(14)
# print(f"length of raw2_lag is {len(raw2_lag)}")
# raw2_lag.tail()

# look at lagging after windowing
raw3_lag = raw1_mov.shift(1)
print(f"length of raw3_lag is {len(raw3_lag)}")
raw3_lag.tail()



tail of raw is:                            c
time                        
2020-06-08 14:20:00  1.13082
2020-06-08 14:30:00  1.12979
2020-06-08 14:40:00  1.12994
2020-06-08 14:50:00  1.13009
2020-06-08 15:00:00  1.13033
length of raw_resample is 15
tail after resampling                            c
time                        
2020-06-08 11:00:00  1.12712
2020-06-08 12:00:00  1.12764
2020-06-08 13:00:00  1.12940
2020-06-08 14:00:00  1.13097
2020-06-08 15:00:00  1.13009
length of raw1 is: 15
length of raw1_mov is 15
data after moving average: 
 time
2020-06-08 11:00:00         NaN
2020-06-08 12:00:00         NaN
2020-06-08 13:00:00         NaN
2020-06-08 14:00:00    1.129240
2020-06-08 15:00:00    1.129264
Freq: H, Name: c, dtype: float64
length of raw3_lag is 15


time
2020-06-08 11:00:00        NaN
2020-06-08 12:00:00        NaN
2020-06-08 13:00:00        NaN
2020-06-08 14:00:00        NaN
2020-06-08 15:00:00    1.12924
Freq: H, Name: c, dtype: float64

In [158]:
class MLTrader(tpqoa.tpqoa):
    def __init__(self, config_file, algorithm):
        # this is simply using method from base class
        # see: https://github.com/yhilpisch/tpqoa/blob/master/tpqoa/tpqoa.py
        super(MLTrader, self).__init__(config_file)
        # all of these can be parameters of the class instantiation
        # but in this simple example, they are hard coded
        # THESE PARAMETERS SHOULD MATCH WHAT YOU USED DURING
        # BACKTESTING!!!
        self.model = algorithm['model']
        self.mu = algorithm['mu']
        self.std = algorithm['std']
        # our trading capital
        self.units = 100000
        # initial position is none
        self.position = 0
        # two second bars
        self.bar = '1s'
        # window from moving averages and min/maxes
        self.window = 20
        # number of lags for lagged features
        self.lags = 6
        # window size snips off (window size - 1) from beginning of data
        # lag forward snipss off (lag) from beginning of data
        # But you have one datapoint with (lags + window) length of data!
        # +1 makes it so that first order is placed after a short while
        self.min_length = self.lags + self.window + 1
        # features to lag
        self.features = ['log_returns', 'vol', 'mom', 'sma', 'min', 'max']
        self.raw_data = pd.DataFrame()
    
    # same as above feature engineering!
    def prepare_features(self):
        self.data['log_returns'] = np.log(
            self.data['mid'] / self.data['mid'].shift(1))
        self.data['vol'] = self.data['log_returns'].rolling(self.window).std()
        self.data['mom'] = np.sign(
            self.data['log_returns'].rolling(self.window).mean())
        self.data['sma'] = self.data['mid'].rolling(self.window).mean()
        self.data['min'] = self.data['mid'].rolling(self.window).min()
        self.data['max'] = self.data['mid'].rolling(self.window).max()
        self.data.dropna(inplace=True)
        # only normalize input features
        self.data[self.features] -= self.mu
        self.data[self.features] /= self.std
        self.cols = []
        for f in self.features:
            for lag in range(1, self.lags + 1):
                col = f'{f}_lag_{lag}'
                self.data[col] = self.data[f].shift(lag)
                self.cols.append(col)

    def on_success(self, time, bid, ask):
        # This on success method essentially is given the streaming
        # bid and ask data from the stream_data method.
        # see https://github.com/yhilpisch/tpqoa/blob/master/tpqoa/tpqoa.py
        print(self.ticks, 20 * ' ', end='\r')
        df = pd.DataFrame({'bid': float(bid), 'ask': float(ask)},
                          index=[pd.Timestamp(time).tz_localize(None)])
        #print(df.tail())
        self.raw_data = pd.concat([self.raw_data, df])
        #print(self.raw_data.tail())
        # the bucket is labelled by the right bin edge
        # the default side of the bin which is closed is the left side
        # (tilt your head to the right and look at the dataframe
        #  and that is the orientation we're talking about)
        # see above validation cells and documentation
        self.data = self.raw_data.resample(
            self.bar, label='right').last().ffill()
        self.data = self.data.iloc[:-1]
        print(f"min_length is: {self.min_length}")
        print(f"length of data is: {len(self.data)}")
        print(self.data.tail())
        if len(self.data) > self.min_length:
            #logger_monitor('NUMBER OF TICKS: {} | '.format(self.ticks) +
            #               'NUMBER OF BARS: {}'.format(self.min_length))
            # incrementing min_length to wait for next resampled bar to come in
            self.min_length += 1
            self.data['mid'] = (self.data['bid'] + self.data['ask']) / 2
            self.prepare_features()
            # take the lagged features, their last value, then reshape.
            # essentially you take the last observation (this becomes a pandas series),
            # you convert it to the numpy array with .values, and then you reshape it 
            # to a 2D array because this is what the model expects
            features = self.data[self.cols].iloc[-1].values.reshape(1, -1)
            #print(features)
            # algo strategy signal
            signal = self.model.predict(features)[0]
            # try simple moving average to see if this is working 
            #print(f"mid price is: {self.data['mid'].tail()}")
            #signal = np.sign(
            #np.log(
            #self.data['mid'] / self.data['mid'].shift(1)).rolling(self.window).mean()).iloc[-1]
            #print(self.data['mom'])
            print(f"Signal is: {signal}")
            # logs and sends major financial information
            #logger_monitor('MOST RECENT DATA\n' +
            #               str(self.data[self.cols].tail()),
            #               False)
            #logger_monitor('features:\n' + str(features) + '\n' +
            #               'position: ' + str(self.position) + '\n' +
            #               'signal:   ' + str(signal), False)
            if self.position in [0, -1] and signal == 1:  # going long?
                # we take (1 - position) units because this accounts for
                # case where we were short (position = -1) so we have to 
                # both close it out and go long (2 * units)
                order = self.create_order(self.stream_instrument,
                                          units=(1 - self.position) *
                                          self.units,
                                          suppress=True, ret=True)
                #self.report_trade('LONG', order)
                print('Long', order)
                self.position = 1
            elif self.position in [0, 1] and signal == -1:  # going short?
                order = self.create_order(self.stream_instrument,
                                          units=-(1 + self.position) *
                                          self.units,
                                          suppress=True, ret=True)
                #self.report_trade('SHORT', order)
                print('Long', order)
                self.position = -1
            else:  # no trade
                print('*** NO TRADE PLACED ***')
                #logger_monitor('*** NO TRADE PLACED ***')
            print('*** END OF CYCLE ***\n\n', False, False)
            #logger_monitor('*** END OF CYCLE ***\n\n', False, False)

In [159]:
mlt = MLTrader('../../data/keys.cfg', algorithm)

In [160]:
# stop after 500 bars!
mlt.stream_data(instrument, stop=500)
print('*** CLOSING OUT ***')
mlt.create_order(mlt.stream_instrument,units=-mlt.position * mlt.units)

min_length is: 27     
length of data is: 0
Empty DataFrame
Columns: [bid, ask]
Index: []
min_length is: 27     
length of data is: 4
                         bid      ask
2022-03-28 02:34:07  1.09568  1.09584
2022-03-28 02:34:08  1.09568  1.09584
2022-03-28 02:34:09  1.09568  1.09584
2022-03-28 02:34:10  1.09568  1.09584
min_length is: 27     
length of data is: 6
                         bid      ask
2022-03-28 02:34:08  1.09568  1.09584
2022-03-28 02:34:09  1.09568  1.09584
2022-03-28 02:34:10  1.09568  1.09584
2022-03-28 02:34:11  1.09569  1.09583
2022-03-28 02:34:12  1.09569  1.09583
min_length is: 27     
length of data is: 8
                         bid      ask
2022-03-28 02:34:10  1.09568  1.09584
2022-03-28 02:34:11  1.09569  1.09583
2022-03-28 02:34:12  1.09569  1.09583
2022-03-28 02:34:13  1.09569  1.09583
2022-03-28 02:34:14  1.09569  1.09583
min_length is: 27     
length of data is: 11
                         bid      ask
2022-03-28 02:34:13  1.09569  1.09583
2022-03-28 

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


Long {'id': '123', 'time': '2022-03-28T02:34:34.922607538Z', 'userID': 21798954, 'accountID': '101-001-21798954-001', 'batchID': '122', 'requestID': '42942834006954515', 'type': 'ORDER_FILL', 'orderID': '122', 'instrument': 'EUR_USD', 'units': '100000.0', 'gainQuoteHomeConversionFactor': '1.0', 'lossQuoteHomeConversionFactor': '1.0', 'price': 1.09584, 'fullVWAP': 1.09584, 'fullPrice': {'type': 'PRICE', 'bids': [{'price': 1.09567, 'liquidity': '10000000'}], 'asks': [{'price': 1.09584, 'liquidity': '10000000'}], 'closeoutBid': 1.09567, 'closeoutAsk': 1.09584}, 'reason': 'MARKET_ORDER', 'pl': '0.0', 'financing': '0.0', 'commission': '0.0', 'guaranteedExecutionFee': '0.0', 'accountBalance': '99699.911', 'tradeOpened': {'tradeID': '123', 'units': '100000.0', 'price': 1.09584, 'guaranteedExecutionFee': '0.0', 'halfSpreadCost': '8.5', 'initialMarginRequired': '2191.52'}, 'halfSpreadCost': '8.5'}
*** END OF CYCLE ***

 False False
min_length is: 28      
length of data is: 28
                 

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


min_length is: 29      
length of data is: 31
                         bid      ask
2022-03-28 02:34:33  1.09567  1.09584
2022-03-28 02:34:34  1.09568  1.09583
2022-03-28 02:34:35  1.09567  1.09584
2022-03-28 02:34:36  1.09567  1.09584
2022-03-28 02:34:37  1.09569  1.09584
Signal is: 1
*** NO TRADE PLACED ***
*** END OF CYCLE ***

 False False


...

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


min_length is: 501      
length of data is: 604
                         bid     ask
2022-03-28 02:44:06  1.09541  1.0956
2022-03-28 02:44:07  1.09541  1.0956
2022-03-28 02:44:08  1.09541  1.0956
2022-03-28 02:44:09  1.09541  1.0956
2022-03-28 02:44:10  1.09541  1.0956
Signal is: 1
*** NO TRADE PLACED ***
*** END OF CYCLE ***

 False False


C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


min_length is: 502      
length of data is: 604
                         bid     ask
2022-03-28 02:44:06  1.09541  1.0956
2022-03-28 02:44:07  1.09541  1.0956
2022-03-28 02:44:08  1.09541  1.0956
2022-03-28 02:44:09  1.09541  1.0956
2022-03-28 02:44:10  1.09541  1.0956
Signal is: 1
*** NO TRADE PLACED ***
*** END OF CYCLE ***

 False False


C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
